In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import ttest_1samp

# Journal Summary
"""
Balanced Risk Set Matching is a method used in observational studies to match treated patients with control patients who have similar medical histories up to the time of treatment. The procedure ensures comparability between groups while minimizing covariate imbalances. 

This study applies integer programming to achieve optimal balanced matching, minimizing covariate distances. Sensitivity analysis evaluates potential biases due to unmeasured variables. The results are analyzed using statistical tests to assess the effect of treatment.
"""

# Simulated Data: Patients with Treatment Time and Symptoms
def generate_sample_data(n=200):
    np.random.seed(42)
    data = pd.DataFrame({
        'patient_id': np.arange(n),
        'treatment_time': np.random.choice([3, 6, 9, 12, np.nan], size=n, p=[0.2, 0.2, 0.2, 0.2, 0.2]),
        'pain': np.random.randint(0, 10, size=n),
        'urgency': np.random.randint(0, 10, size=n),
        'frequency': np.random.randint(0, 10, size=n)
    })
    return data

data = generate_sample_data()
print(data.head())

# Risk Set Matching: Finding Controls for Treated Patients
def risk_set_matching(data):
    treated = data.dropna(subset=['treatment_time'])
    controls = data[data['treatment_time'].isna()]
    
    # Compute pairwise distances based on symptoms
    dist_matrix = cdist(treated[['pain', 'urgency', 'frequency']], 
                        controls[['pain', 'urgency', 'frequency']], metric='euclidean')
    
    # Solve assignment problem (Hungarian Algorithm)
    row_ind, col_ind = linear_sum_assignment(dist_matrix)
    matched_controls = controls.iloc[col_ind].reset_index(drop=True)
    matched_treated = treated.iloc[row_ind].reset_index(drop=True)
    
    return matched_treated, matched_controls

matched_treated, matched_controls = risk_set_matching(data)

# Performing a Simple Paired t-Test
def analyze_results(matched_treated, matched_controls):
    diff = matched_treated[['pain', 'urgency', 'frequency']].values - matched_controls[['pain', 'urgency', 'frequency']].values
    t_stat, p_value = ttest_1samp(diff, 0, axis=0)
    return t_stat, p_value

results = analyze_results(matched_treated, matched_controls)
print("T-test statistics:", results)

# Visualization
plt.figure(figsize=(12, 5))
plt.subplot(1, 3, 1)
plt.boxplot([matched_treated['pain'], matched_controls['pain']], labels=['Treated', 'Control'])
plt.title('Pain Scores')

plt.subplot(1, 3, 2)
plt.boxplot([matched_treated['urgency'], matched_controls['urgency']], labels=['Treated', 'Control'])
plt.title('Urgency Scores')

plt.subplot(1, 3, 3)
plt.boxplot([matched_treated['frequency'], matched_controls['frequency']], labels=['Treated', 'Control'])
plt.title('Frequency Scores')

plt.suptitle('Comparison of Matched Groups')
plt.show()
